# Nuisance Alarm Filter

## Documentation

lorem ipsum

## Initialisation

### Load libraries

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import datetime as dt
import time

### Configure display

In [2]:
# Enable display of all columns for dataframes with many variables
pd.set_option('display.max_columns', None)

### Set Up Directory

In [3]:
# Check current directory location
import os
cwd = os.getcwd()
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\scripts'

In [4]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + alarmLoc)
os.chdir(os.path.dirname(os.getcwd()) + '\\alarm-event-logs')

# Check current directory location
cwd = os.getcwd()

# Check directory location
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [5]:
# Location of Alarm and Normal Event Files
alarmLoc = '\\sample_data_ecs\\batch 001 - 20201230 to 20210201\\dataCleaned\\AlarmLog_meta'
eventLoc = '\\sample_data_ecs\\batch 001 - 20201230 to 20210201\\dataCleaned\\EventLog_meta'
#alarmLoc = '\\sample_data_ats\\batch 001 - 20201230 to 20210201\\dataCleaned\\AlarmLog_meta'
#eventLoc = '\\sample_data_ats\\batch 001 - 20201230 to 20210201\\dataCleaned\\EventLog_meta'
#alarmLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\dataCleaned\\AlarmLog_meta'
#eventLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\dataCleaned\\EventLog_meta'

## Load EventList Files

In [6]:
# Check directory location
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [7]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + alarmLoc)
os.chdir(cwd + alarmLoc)
# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_ecs\\batch 001 - 20201230 to 20210201\\dataCleaned\\AlarmLog_meta'

In [8]:
# Gather list of cleaned alarmList meta data files
dfs = [pd.read_csv(f)
        for f in os.listdir(os.getcwd()) if f.endswith('csv')]

print(len(dfs))

# Compile list of cleaned alarmList files into a single dataframe
df = pd.concat(dfs, ignore_index=True).reset_index()

# Configure data type of date value
df['DATETIME_RECEIVED'] = pd.to_datetime(df['DATETIME_RECEIVED'])

# Sort values by datetime
df = df.sort_values(by = 'DATETIME_RECEIVED', ascending = True)

# Delete redundant index col
del df["index"]

# Print df summary stats
print(df.shape)
print(df.info())

11
(523236, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 523236 entries, 434489 to 484488
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   FileName           523236 non-null  object        
 1   FileSize_Bytes     523236 non-null  int64         
 2   DATETIME_RECEIVED  523236 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 16.0+ MB
None


In [9]:
# Inspect df
df.head()

,FileName,FileSize_Bytes,DATETIME_RECEIVED
434489,AlarmListTest_1608663455_838030,100,2020-12-23 02:57:35.838
434490,AlarmListTest_1608663457_011107,10786,2020-12-23 02:57:37.011
434491,AlarmListTest_1608663463_417482,111111,2020-12-23 02:57:43.417
434492,AlarmListTest_1608663465_435941,1127,2020-12-23 02:57:45.435
434493,AlarmListTest_1608663466_343256,8600,2020-12-23 02:57:46.343


## Perform Rolling Window Summary

#### Reindex Data By Time

In [10]:
# Create copy of time sequence field (DATETIME_RECEIVED) as a proto index column
df["datetimeIndex"] = df["DATETIME_RECEIVED"].copy()

# Set Date-Time to be the index
df = df.set_index(df["datetimeIndex"])

# Drop proto index column
del df["datetimeIndex"]

# Inspect data sample
df.head()

,FileName,FileSize_Bytes,DATETIME_RECEIVED
datetimeIndex,,,
2020-12-23 02:57:35.838,AlarmListTest_1608663455_838030,100,2020-12-23 02:57:35.838
2020-12-23 02:57:37.011,AlarmListTest_1608663457_011107,10786,2020-12-23 02:57:37.011
2020-12-23 02:57:43.417,AlarmListTest_1608663463_417482,111111,2020-12-23 02:57:43.417
2020-12-23 02:57:45.435,AlarmListTest_1608663465_435941,1127,2020-12-23 02:57:45.435
2020-12-23 02:57:46.343,AlarmListTest_1608663466_343256,8600,2020-12-23 02:57:46.343


### Perform Rolling Window Count of Files

In [11]:
# Count number of files within window period
df["fileCount_01sec"] = df["FileName"].rolling('1s').count()
df["fileCount_60sec"] = df["FileName"].rolling('60s').count()

# Inspect data sample
df.head()

,FileName,FileSize_Bytes,DATETIME_RECEIVED,fileCount_01sec,fileCount_60sec
datetimeIndex,,,,,
2020-12-23 02:57:35.838,AlarmListTest_1608663455_838030,100,2020-12-23 02:57:35.838,1.0,1.0
2020-12-23 02:57:37.011,AlarmListTest_1608663457_011107,10786,2020-12-23 02:57:37.011,1.0,2.0
2020-12-23 02:57:43.417,AlarmListTest_1608663463_417482,111111,2020-12-23 02:57:43.417,1.0,3.0
2020-12-23 02:57:45.435,AlarmListTest_1608663465_435941,1127,2020-12-23 02:57:45.435,1.0,4.0
2020-12-23 02:57:46.343,AlarmListTest_1608663466_343256,8600,2020-12-23 02:57:46.343,2.0,5.0


In [12]:
# Sum total file size within window period
df["fileSize_Bytes_01sec"] = df["FileSize_Bytes"].rolling('1s').sum()
df["fileSize_Bytes_60sec"] = df["FileSize_Bytes"].rolling('60s').sum()

# Inspect data sample
df.head()

,FileName,FileSize_Bytes,DATETIME_RECEIVED,fileCount_01sec,fileCount_60sec,fileSize_Bytes_01sec,fileSize_Bytes_60sec
datetimeIndex,,,,,,,
2020-12-23 02:57:35.838,AlarmListTest_1608663455_838030,100,2020-12-23 02:57:35.838,1.0,1.0,100.0,100.0
2020-12-23 02:57:37.011,AlarmListTest_1608663457_011107,10786,2020-12-23 02:57:37.011,1.0,2.0,10786.0,10886.0
2020-12-23 02:57:43.417,AlarmListTest_1608663463_417482,111111,2020-12-23 02:57:43.417,1.0,3.0,111111.0,121997.0
2020-12-23 02:57:45.435,AlarmListTest_1608663465_435941,1127,2020-12-23 02:57:45.435,1.0,4.0,1127.0,123124.0
2020-12-23 02:57:46.343,AlarmListTest_1608663466_343256,8600,2020-12-23 02:57:46.343,2.0,5.0,9727.0,131724.0


## Print Summary Stats

In [13]:
# Generate Summary Stats
df.describe()

,FileSize_Bytes,fileCount_01sec,fileCount_60sec,fileSize_Bytes_01sec,fileSize_Bytes_60sec
count,523236.000000,523236.000000,523236.000000,523236.000000,5.232360e+05
mean,794.668456,1.106065,12.127184,904.614906,1.050424e+04
std,988.605176,0.356091,6.807775,1334.712296,1.562253e+04
min,100.000000,1.000000,1.000000,100.000000,1.000000e+02
25%,629.000000,1.000000,8.000000,629.000000,5.555000e+03
50%,631.000000,1.000000,11.000000,631.000000,7.786000e+03
75%,639.000000,1.000000,14.000000,1126.000000,1.094700e+04
max,250715.000000,4.000000,60.000000,256258.000000,1.005740e+06


In [14]:
relevantCol = df.select_dtypes(['int64', 'float']).columns
print("SUMMARY STATS")
print()
print("===============")
print()
n = 0
while n < len(relevantCol):
    print(relevantCol[n])
    print("Mean:", str(round(df[relevantCol[n]].mean(), 5)))
    print("25th Percentile:", str(round(df[relevantCol[n]].quantile(.25), 5)))
    print("Median:", str(round(df[relevantCol[n]].median(), 5)))
    print("75th Percentile:", str(round(df[relevantCol[n]].quantile(.75), 5)))
    print("90th Percentile:", str(round(df[relevantCol[n]].quantile(.95), 5)))
    print("95th Percentile:", str(round(df[relevantCol[n]].quantile(.95), 5)))
    print("Min:", str(round(df[relevantCol[n]].min(), 5)))
    print("Max:", str(round(df[relevantCol[n]].max(), 5)))
    print()
    print("===============")
    print()
    n = n + 1

SUMMARY STATS


FileSize_Bytes
Mean: 794.66846
25th Percentile: 629.0
Median: 631.0
75th Percentile: 639.0
90th Percentile: 1151.0
95th Percentile: 1151.0
Min: 100
Max: 250715


fileCount_01sec
Mean: 1.10606
25th Percentile: 1.0
Median: 1.0
75th Percentile: 1.0
90th Percentile: 2.0
95th Percentile: 2.0
Min: 1.0
Max: 4.0


fileCount_60sec
Mean: 12.12718
25th Percentile: 8.0
Median: 11.0
75th Percentile: 14.0
90th Percentile: 27.0
95th Percentile: 27.0
Min: 1.0
Max: 60.0


fileSize_Bytes_01sec
Mean: 904.61491
25th Percentile: 629.0
Median: 631.0
75th Percentile: 1126.0
90th Percentile: 1784.0
95th Percentile: 1784.0
Min: 100.0
Max: 256258.0


fileSize_Bytes_60sec
Mean: 10504.24461
25th Percentile: 5555.0
Median: 7786.0
75th Percentile: 10947.0
90th Percentile: 29434.0
95th Percentile: 29434.0
Min: 100.0
Max: 1005740.0




In [15]:
# Inspect data structure
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 523236 entries, 2020-12-23 02:57:35.838000 to 2021-02-01 02:18:11.502000
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   FileName              523236 non-null  object        
 1   FileSize_Bytes        523236 non-null  int64         
 2   DATETIME_RECEIVED     523236 non-null  datetime64[ns]
 3   fileCount_01sec       523236 non-null  float64       
 4   fileCount_60sec       523236 non-null  float64       
 5   fileSize_Bytes_01sec  523236 non-null  float64       
 6   fileSize_Bytes_60sec  523236 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 31.9+ MB
